## Microsoft Location Mention Recognition Challenge by Team_O2

In [1]:
# Import the necesssary Libraries and Dependencies
import pandas as pd
import json
import os
import math
import re
import numpy as np
import torch
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda

# Set the maximum width of pandas DataFrame to 1000
pd.set_option('display.max_columns', 1000)

In [ ]:
# types = {"Island": "ISL", "State": "STAT", "Continent": "CONT", "City/town": "CITY", "Country": "CTRY",
#         "County": "CNTY", "Neighborhood": "NBHD", "Road/street": "ST", "District": "DIST", "Other locations": "OTHR", 
#         "Natural Point-of-Interest": "NPOI", "Human-made Point-of-Interest": "HPOI"}

# en_events = ["california_wildfires_2018", "canada_wildfires_2016", "cyclone_idai_2019", "ecuador_earthquake_2016",
#              "greece_wildfires_2018", "hurricane_dorian_2019", "hurricane_florence_2018", "hurricane_harvey_2017",
#              "hurricane_irma_2017", "hurricane_maria_2017", "hurricane_matthew_2016", "italy_earthquake_aug_2016",
#              "kaikoura_earthquake_2016", "kerala_floods_2018", "maryland_floods_2018", "midwestern_us_floods_2019",
#              "pakistan_earthquake_2019", "puebla_mexico_earthquake_2017", "srilanka_floods_2017"]

In [3]:
# Load data
train_data = pd.read_json('./Data/california_wildfires_2018/train.jsonl', lines=True)
test_data = pd.read_json('./Data/california_wildfires_2018/test_unlabeled.jsonl', lines=True)
dev_data = pd.read_json('./Data/california_wildfires_2018/dev.jsonl', lines=True)


In [5]:
# Initialize an empty list to store the data
train_df = []

# Open the JSON Lines file and read it line by line
with open('./Data/california_wildfires_2018/train.jsonl', 'r') as file:
    for line in file:
        train_df.append(json.loads(line))

# Convert the list of dictionaries into a DataFrame
df = pd.DataFrame(train_df)


In [8]:
# Initialize an empty list to store the data
dev_df = []

# Open the JSON Lines file and read it line by line
with open('./Data/california_wildfires_2018/dev.jsonl', 'r') as file:
    for line in file:
        dev_df.append(json.loads(line))

# Convert the list of dictionaries into a DataFrame
df2 = pd.DataFrame(dev_df)


In [9]:
df2.head()

,tweet_id,user_id,text,created_at,humAID_class,location_mentions
0,1065347154078773250,228351372,If youre looking for legitimate relief organiz...,Wed Nov 21 20:52:09 +0000 2018,rescue_volunteering_or_donation_effort,"[{'text': 'CA', 'type': 'State', 'start_offset..."
1,1065256190727282689,2312720533,"Like so many things, the destruction of Califo...",Wed Nov 21 14:50:42 +0000 2018,infrastructure_and_utility_damage,"[{'text': 'Californias', 'type': 'State', 'sta..."
2,1066000788177211393,16138559,Officials say 563 people are still unaccounted...,Fri Nov 23 16:09:27 +0000 2018,injured_or_dead_people,[]
3,1064315981365862400,225245091,"The wait for Paradise: From young to old, evac...",Mon Nov 19 00:34:38 +0000 2018,displaced_people_and_evacuations,"[{'text': 'Chico', 'type': 'City/town', 'start..."
4,1067241739843325952,853466688926261248,BREAKING: Camp Fire death toll increases to 88...,Tue Nov 27 02:20:33 +0000 2018,injured_or_dead_people,[]


In [12]:
df2.shape

(130, 6)

In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   tweet_id           130 non-null    object
 1   user_id            130 non-null    object
 2   text               130 non-null    object
 3   created_at         130 non-null    object
 4   humAID_class       130 non-null    object
 5   location_mentions  130 non-null    object
dtypes: object(6)
memory usage: 6.2+ KB


In [6]:
# Display the first few rows of the DataFrame
df.head()


,tweet_id,user_id,text,created_at,humAID_class,location_mentions
0,1061497252806414336,886453870120976384,Please read below!! Another devastating fire h...,Sun Nov 11 05:54:01 +0000 2018,requests_or_urgent_needs,"[{'text': 'Northern California', 'type': 'Stat..."
1,1061165982855634944,301309392,Celebrities evacuate California as wildfires rage,Sat Nov 10 07:57:40 +0000 2018,displaced_people_and_evacuations,"[{'text': 'California', 'type': 'State', 'star..."
2,1064248785914732544,973902672083419139,Grab one of these and HELP victims of Californ...,Sun Nov 18 20:07:38 +0000 2018,rescue_volunteering_or_donation_effort,"[{'text': 'California', 'type': 'State', 'star..."
3,1061160485364613121,412693014,the camp fire in Paradise California is growin...,Sat Nov 10 07:35:49 +0000 2018,missing_or_found_people,"[{'text': 'Paradise', 'type': 'City/town', 'st..."
4,1066508972650311682,818643843763568644,The area of documented destruction in the #Cam...,Sun Nov 25 01:48:48 +0000 2018,infrastructure_and_utility_damage,[]


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130 entries, 0 to 129
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   tweet_id           130 non-null    object
 1   user_id            130 non-null    object
 2   text               130 non-null    object
 3   created_at         130 non-null    object
 4   humAID_class       130 non-null    object
 5   location_mentions  130 non-null    object
dtypes: object(6)
memory usage: 6.2+ KB


In [11]:
df.shape

(130, 6)

In [6]:
# Replace missing values with empty strings  
df = df.fillna('')  

In [8]:
df.isna().sum()

tweet_id             0
user_id              0
text                 0
created_at           0
humAID_class         0
location_mentions    0
dtype: int64

In [16]:
# import nltk
# from nltk.tokenize import word_tokenize

# # Download the NLTK tokenizer resources
# nltk.download('punkt')

In [14]:
# from nltk.data import find

# try:
#     find('tokenizers/punkt')
#     print("Punkt tokenizer is available.")
# except LookupError:
#     print("Punkt tokenizer is not found.")


Punkt tokenizer is available.


In [15]:
# # Tokenize the 'text' column
# df['tokenized_text'] = df['text'].apply(word_tokenize)


In [ ]:
import re

# Define a function to clean text
def clean_text(text):
    text = re.sub(r'http\S+|www\S+|https\S+', '', text, flags=re.MULTILINE)  # Remove URLs
    text = re.sub(r'@\w+', '', text)  # Remove mentions
    text = re.sub(r'#\w+', '', text)  # Remove hashtags
    text = re.sub(r'[^\w\s]', '', text)  # Remove punctuation
    return text

# Apply the cleaning function
df['cleaned_text'] = df['text_column'].apply(clean_text)
